# Rodar Teste 1: influência da SNR

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from typing import Optional, Any
from pathlib import Path
from time import time
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from joblib import Parallel, delayed

## Funções

In [2]:
def abrirDataframe(
    dirDatasets:        Path, 
    qtdUsuarios:        int, 
    qtdAntenas:         int,
    datasetTreinamento: bool
) -> pd.DataFrame:
    
    prefixo    = "train_1_" if datasetTreinamento else "test_1_"        
    caminhoCSV = sorted(dirDatasets.glob(prefixo+str(qtdUsuarios)+"_users_"+str(qtdAntenas)+"_antenas_*.csv"))[-1]
    df         = pd.read_csv(caminhoCSV)
    
    return df

In [3]:
def separarXeY(
    dfTreinamento:        pd.DataFrame,
    dfTeste:              pd.DataFrame, 
    featuresSelecionadas: Optional[list[str]] = None
) -> (pd.Series, pd.Series, pd.Series, pd.Series, pd.Series, pd.Series, pd.Series, pd.Series):
    
    # SELECIONANDO TODAS AS FEATURES SE FEATURESSELECIONADAS VIER NONE
    if featuresSelecionadas is None:
        featuresSelecionadas = df.columns.drop(["potenciaEspiao", "ataquePresente"])
    
    # SEPARANDO OS ARRAYS
    XTrain         = dfTreinamento[featuresSelecionadas]
    XTest          = dfTeste[featuresSelecionadas]
    yTrain         = dfTreinamento["ataquePresente"]
    yTest          = dfTeste["ataquePresente"]
    snrTrain       = dfTreinamento["snr"]
    snrTest        = dfTeste["snr"]
    potEspiaoTrain = dfTreinamento["potenciaEspiao"]
    potEspiaoTest  = dfTeste["potenciaEspiao"]
    
    return XTrain, XTest, yTrain, yTest, snrTrain, snrTest, potEspiaoTrain, potEspiaoTest

In [4]:
def deteccaoMachineLearning(
    XTrain:           pd.Series, 
    XTest:            pd.Series, 
    yTrain:           pd.Series, 
    objClassificador: Any
) -> (np.ndarray, np.float64, np.float64):

    # TREINANDO O CLASSIFICADOR
    objClassificador.fit(XTrain, yTrain)

    # PREDIZENDO OS DE TESTE
    arrayTempos = np.zeros(len(XTest))
    yPred       = np.zeros(len(XTest))
    for i, (id, amostraTeste) in enumerate(XTest.iterrows()):
        tic            = time()
        yPred[i]       = objClassificador.predict([amostraTeste.to_numpy()])[0]
        toc            = time()
        arrayTempos[0] = toc-tic
        
    # METRICAS DE TEMPO
    mediaTempo, desvioTempo = np.mean(arrayTempos), np.std(arrayTempos)
        
    return yPred, mediaTempo, desvioTempo

In [5]:
def deteccaoHassan(
    XTest: pd.Series
) -> (np.ndarray, np.float64, np.float64):
    
    arrayTempos = np.zeros(len(XTest))
    yPred       = np.zeros(len(XTest))
    
    for i, (id, amostraTeste) in enumerate(XTest.iterrows()):
        tic            = time()
        yPred[i]       = 1 if amostraTeste["E"] > amostraTeste["eta"] else 0
        toc            = time()
        arrayTempos[i] = toc-tic

    mediaTempo, desvioTempo = np.mean(arrayTempos), np.std(arrayTempos)
        
    return yPred, mediaTempo, desvioTempo

In [6]:
def deteccaoKapetanovic(
    XTest:     pd.Series, 
    threshold: float
) -> (np.ndarray, np.float64, np.float64):
    
    arrayTempos = np.zeros(len(XTest))
    yPred       = np.zeros(len(XTest))
    
    for i, (id, amostraTeste) in enumerate(XTest.iterrows()):
        tic            = time()
        yPred[i]       = 1 if amostraTeste["a1"]/amostraTeste["a2"] > threshold else 0
        toc            = time()
        arrayTempos[i] = toc-tic
        
    mediaTempo, desvioTempo = np.mean(arrayTempos), np.std(arrayTempos)
        
    return yPred, mediaTempo, desvioTempo

In [7]:
def predizerEsquemaDeteccao(
    esquemaDeteccao:      str,
    XTrain:               pd.Series, 
    XTest:                pd.Series, 
    yTrain:               pd.Series, 
    classificador:        Any,
    thresholdKapetanovic: float
) -> (np.ndarray, np.float64, np.float64):
    
    if esquemaDeteccao == "ML":
        yPred, tempoMedioPredicao, tempoDesvioPredicao = deteccaoMachineLearning(XTrain, XTest, yTrain, classificador)
    elif esquemaDeteccao == "Hassan":
        yPred, tempoMedioPredicao, tempoDesvioPredicao = deteccaoHassan(XTest)
    else:
        yPred, tempoMedioPredicao, tempoDesvioPredicao = deteccaoKapetanovic(XTest, thresholdKapetanovic)
        
    return yPred, tempoMedioPredicao, tempoDesvioPredicao

In [8]:
def obterValoresUnicos(
    arraySNRs:       np.ndarray, 
    arrayPotsEspiao: np.ndarray
) -> (np.ndarray, np.ndarray):
    
    rangeSNRs      = np.unique(arraySNRs)
    rangePotEspiao = np.unique(arrayPotsEspiao)
    
    return rangeSNRs, rangePotEspiao

In [9]:
def calcularMetricas(
    matrizProbabilidadeDeteccaoCadaKFold: list[np.ndarray],
    matrizConfusaoCadaKFold:              list[np.ndarray],
    acuraciaCadaKFold:                    list[np.float64],
    precisaoCadaKFold:                    list[np.float64],
    revocacaoCadaKFold:                   list[np.float64],
    tempoProcessamentoCadaKFold:          list[np.float64],
    rangePotEspiao:                       np.ndarray,
    rangeSNRs:                            np.ndarray
) -> dict[str, Any]:
    
    # MATRIZ DE PROBABILIDADE DE DETECCAO MEDIA E DESVIO EM CADA POT ESP E CADA SNR
    matrizProbabilidadeDeteccaoMedia  = np.mean(matrizProbabilidadeDeteccaoCadaKFold, axis=0)
    matrizProbabilidadeDeteccaoDesvio = np.std(matrizProbabilidadeDeteccaoCadaKFold, axis=0)

    # MATRIZ DE CONFUSAO GERAL (SOMA DE TODAS AS ITERACOES)
    somaMatrizesConfusao = np.sum(matrizConfusaoCadaKFold, axis=0)

    # MEDIA E DESVIO DE OUTRAS METRICAS
    acuraciaMedia, acuraciaDesvio                      = np.mean(acuraciaCadaKFold), np.std(acuraciaCadaKFold)
    precisaoMedia, precisaoDesvio                      = np.mean(precisaoCadaKFold), np.std(precisaoCadaKFold)
    revocacaoMedia, revocacaoDesvio                    = np.mean(revocacaoCadaKFold), np.std(revocacaoCadaKFold)
    tempoProcessamentoMedio, tempoProcessamentoDesvio  = np.mean(tempoProcessamentoCadaKFold), np.std(tempoProcessamentoCadaKFold)
    
    # MONTANDO O DICIONARIO COM AS METRICAS
    dictMetricas = {
        "rangePotEspiao":                    rangePotEspiao.tolist(),
        "rangeSNRs":                         rangeSNRs.tolist(),
        "acuraciaMedia":                     float(acuraciaMedia),
        "acuraciaDesvio":                    float(acuraciaDesvio),
        "precisaoMedia":                     float(precisaoMedia),
        "precisaoDesvio":                    float(precisaoDesvio),
        "revocacaoMedia":                    float(revocacaoMedia),
        "revocacaoDesvio":                   float(revocacaoDesvio),
        "tempoProcessamentoMedio":           float(tempoProcessamentoMedio),
        "tempoProcessamentoDesvio":          float(tempoProcessamentoDesvio),
        "somaMatrizesConfusao":              somaMatrizesConfusao.tolist(),
        "matrizProbabilidadeDeteccaoMedia":  matrizProbabilidadeDeteccaoMedia.tolist(),
        "matrizProbabilidadeDeteccaoDesvio": matrizProbabilidadeDeteccaoDesvio.tolist()
    }
    
    return dictMetricas

In [10]:
def salvarResultados(
    prefixoKeyDict:        str, 
    qtdUsuarios:           int, 
    qtdAntenas:            int, 
    qtdSimbolos:           int,
    repetibilidadeDataset: int,
    qtdFolders:            int, 
    featuresSelecionadas:  list[str],
    dictMetricas:          dict[str, Any],
    arquivoSalvar:         Path
) -> None:
    
    # KEY DO JSON QUE VAI GUARDAR ESSE RESULTADO
    key = prefixoKeyDict + "_" + str(time()).replace(".", "")
    
    # INICIANDO O DICIONARIO DE RESULTADOS
    dictResultado = {
        key: {
            "qtdUsuarios":                 qtdUsuarios,
            "qtdAntenas":                  qtdAntenas,
            "qtdSimbolos":                 qtdSimbolos,
            "repetibilidadeDataset":       repetibilidadeDataset,
            "qtdFolders":                  qtdFolders,
            "featuresSelecionadas":        featuresSelecionadas
        }
    }
    
    # CONCATENANDO COM O DICIONARIO DE METRICAS MEDIAS E DESVIOS
    dictResultado[key] |= dictMetricas
    
    # SALVANDO
    with open(arquivoSalvar, mode="r+") as file:
        dictJSON  = json.load(file)
        dictJSON |= dictResultado
        file.seek(0)
        json.dump(dictJSON, file, indent=4)
        file.truncate()

In [11]:
def rodarTesteDeteccao(
    esquemaDeteccao:      str,
    featuresSelecionadas: list[str],
    dirDatasets:          Path, 
    qtdUsuarios:          int, 
    qtdAntenas:           int,
    qtdFolders:           int,
    classificador:        Any, 
    thresholdKapetanovic: float
) -> dict[str, Any]:

    # ARRAYS QUE VAO GUARDAR OS RESULTADOS DE CADA ITERACAO DO KFOLD
    matrizProbabilidadeDeteccaoCadaKFold = []
    acuraciaCadaKFold                    = []
    precisaoCadaKFold                    = []
    revocacaoCadaKFold                   = []
    matrizConfusaoCadaKFold              = []
    tempoProcessamentoCadaKFold          = []

    # ABRINDO OS DATAFRAMES DE TREINAMENTO E TESTE E SEPARANDO X E Y
    dfTreinamento = abrirDataframe(dirDatasets, qtdUsuarios, qtdAntenas, datasetTreinamento=True)
    dfTeste       = abrirDataframe(dirDatasets, qtdUsuarios, qtdAntenas, datasetTreinamento=False)
    XTrainGlobal, XTestGlobal, yTrainGlobal, yTestGlobal, snrTrainGlobal, snrTestGlobal, potEspTrainGlobal, potEspTestGlobal = separarXeY(dfTreinamento, dfTeste, featuresSelecionadas)

    # VALORES UNICOS DE SNR E POTENCIA DE ESPIAO PRA EU SABER O INDEX DAS MATRIZES DE RESULTADOS
    rangeSNRs, rangePotEspiao = obterValoresUnicos(snrTestGlobal, potEspTestGlobal)

    # OBTENDO INDICES DE TREINAMENTO E TESTE COM KFOLD (NAO DA PRA USAR DO JEITO FACIL PQ SAO DOIS DATAFRAMES DIFERENTES)
    objKFold = KFold(n_splits=qtdFolders, shuffle=True)
    pastasTreinamento, _ = zip(*objKFold.split(XTrainGlobal))
    _, pastasTeste       = zip(*objKFold.split(XTestGlobal))

    # RODANDO O KFOLD
    for indexesTreinamento, indexesTeste in zip(pastasTreinamento, pastasTeste):

        # SEPARANDO X, Y, SNRs E POTENCIAS DO ESPIAO DE TREINAMENTO E TESTE
        XTrain, XTest           = XTrainGlobal.loc[indexesTreinamento], XTestGlobal.loc[indexesTeste]
        yTrain, yTest           = yTrainGlobal.loc[indexesTreinamento], yTestGlobal.loc[indexesTeste]
        snrTrain, snrTest       = snrTrainGlobal.loc[indexesTreinamento], snrTestGlobal.loc[indexesTeste]
        potEspTrain, potEspTest = potEspTrainGlobal.loc[indexesTreinamento], potEspTestGlobal.loc[indexesTeste]

        # PREDICAO
        yPred, tempoMedioPredicao, _ = predizerEsquemaDeteccao(esquemaDeteccao, XTrain, XTest, yTrain, classificador, thresholdKapetanovic)
        
        # GRAVANDO METRICAS
        acuraciaCadaKFold.append(accuracy_score(yTest, yPred))                    
        precisaoCadaKFold.append(precision_score(yTest, yPred))                    
        revocacaoCadaKFold.append(recall_score(yTest, yPred))
        matrizConfusaoCadaKFold.append(confusion_matrix(yTest, yPred))
        tempoProcessamentoCadaKFold.append(tempoMedioPredicao)          

        # AGREGO NAS MATRIZES CONTADORAS
        matrizContadoraDeteccoes   = np.zeros(shape=(len(rangePotEspiao), len(rangeSNRs)))
        matrizContadoraOcorrencias = np.zeros(matrizContadoraDeteccoes.shape)
        for (indexTesteAtual, predicaoAtual) in zip(yTest.index, yPred):

            # PEGANDO EM QUAL LINHA E EM QUAL COLUNA DA MATRIZ EU VOU COLOCAR O RESULTADO
            indexPotEsp = np.where(rangePotEspiao==potEspTest[indexTesteAtual])[0][0]
            indexSNR    = np.where(rangeSNRs==snrTest[indexTesteAtual])[0][0]

            # COLOCO NAS MATRIZES DE RESULTADOS
            matrizContadoraDeteccoes[indexPotEsp][indexSNR]   += predicaoAtual
            matrizContadoraOcorrencias[indexPotEsp][indexSNR] += 1

        # MATRIZ DA PROBABILIDADE DE DETECCAO EM CADA POT ESPIAO (LINHAS) E CADA SNR (COLUNAS)
        matrizProbabilidadeDeteccaoCadaKFold.append(matrizContadoraDeteccoes/matrizContadoraOcorrencias)

    # CALCULANDO TODAS AS MEDIAS E DESVIOS DAS METRICAS AO LONGO DO KFOLD E COLOCANDO NUM DICIONARIO
    dictMetricas = calcularMetricas(matrizProbabilidadeDeteccaoCadaKFold, matrizConfusaoCadaKFold, acuraciaCadaKFold, precisaoCadaKFold, revocacaoCadaKFold, tempoProcessamentoCadaKFold, rangePotEspiao, rangeSNRs)

    return dictMetricas

## Parâmetros iniciais

In [19]:
# PARAMETROS
classificador                   = DecisionTreeClassifier()
qtdUsuarios                     = 64
qtdAntenas                      = 256
qtdSimbolos                     = 300
repetibilidadeDataset           = 100
qtdFolders                      = 10
nucleosProcessador              = -1
thresholdKapetanovic            = 100
verbose                         = 100
featuresSelecionadasML          = ["E"]
featuresSelecionadasHassan      = ["E", "eta"]
featuresSelecionadasKapetanovic = ["a1", "a2"]
dirDatasets                     = Path("../../datasets/test_1/")

## Iniciando o JSON qe vai guardar os resultados

In [ ]:
# INICIANDO O JSON QUE VAI GUARDAR OS RESULTADOS
dictResultados = {}
arquivoSalvar  = Path("../../results/test_1/detection_probability_" + str(time()).replace(".", "") + ".json")
with open(arquivoSalvar, mode="w") as file:
    json.dump(dictResultados, file)

## Rodando os testes de detecção com machine learning, hassan e kapetanovic

In [13]:
# PARAMETROS INICIAIS
prefixoKeyDict       = classificador.__class__.__name__
esquemaDeteccao      = "ML"
featuresSelecionadas = featuresSelecionadasML

# RODANDO O TESTE E CALCULANDO METRICAS
dictMetricas = rodarTesteDeteccao(esquemaDeteccao, featuresSelecionadas, dirDatasets, qtdUsuarios, qtdAntenas, qtdFolders, classificador, thresholdKapetanovic)

# SALVANDO O RESULTADO
salvarResultados(prefixoKeyDict, qtdUsuarios, qtdAntenas, qtdSimbolos, repetibilidadeDataset, qtdFolders, featuresSelecionadas, dictMetricas, arquivoSalvar)

In [14]:
# PARAMETROS INICIAIS
prefixoKeyDict       = "Hassan"
esquemaDeteccao      = prefixoKeyDict
featuresSelecionadas = featuresSelecionadasHassan

# RODANDO O TESTE E CALCULANDO METRICAS
dictMetricas = rodarTesteDeteccao(esquemaDeteccao, featuresSelecionadas, dirDatasets, qtdUsuarios, qtdAntenas, qtdFolders, classificador, thresholdKapetanovic)

# SALVANDO O RESULTADO
salvarResultados(prefixoKeyDict, qtdUsuarios, qtdAntenas, qtdSimbolos, repetibilidadeDataset, qtdFolders, featuresSelecionadas, dictMetricas, arquivoSalvar)

In [20]:
# PARAMETROS INICIAIS
prefixoKeyDict       = "Kapetanovic"
esquemaDeteccao      = prefixoKeyDict
featuresSelecionadas = featuresSelecionadasKapetanovic

# RODANDO O TESTE E CALCULANDO METRICAS
dictMetricas = rodarTesteDeteccao(esquemaDeteccao, featuresSelecionadas, dirDatasets, qtdUsuarios, qtdAntenas, qtdFolders, classificador, thresholdKapetanovic)

# SALVANDO O RESULTADO
salvarResultados(prefixoKeyDict, qtdUsuarios, qtdAntenas, qtdSimbolos, repetibilidadeDataset, qtdFolders, featuresSelecionadas, dictMetricas, arquivoSalvar)

/home/dimi/.pyenv/versions/3.9.1/envs/venv_aicom/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dimi/.pyenv/versions/3.9.1/envs/venv_aicom/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dimi/.pyenv/versions/3.9.1/envs/venv_aicom/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dimi/.pyen